# Using Bidirectional Generative Adversarial Networks to estimate Value-at-Risk for Market Risk Management

---

We will explore the use of Bidirectional Generative Adversarial Networks (BiGAN) for market risk management: Estimation of portfolio risk measures such as Value-at-Risk (VaR). Generative Adversarial Networks (GAN) allow us to implicitly maximize the likelihood of complex distributions thereby allowing us to generate samples from such distributions - the key point here is the implicit maximum likelihood estimation principle whereby we do not specify what this complex distribution is parameterized as. Dealing with high dimensional data potentially coming from a complex distribution is a key aspect to market risk management among many other financial services use cases. GAN, specifically BiGAN for the purpose of this paper, will allow us to deal with potentially complex financial services data such that we do not have to explicitly specify a distribution such as a multidimensional Gaussian distribution.

## Market Risk Management: Value-at-Risk (VaR)

---

VaR is a measure of portfolio risk. For instance, a 1% VaR of -5% means that there is a 1% chance of earning a portfolio return of less than -5%. Think of it as a (lower) percentile or quantile of a portfolio returns distribution, i.e., we are concerned about the tail risk — the small chance of losing a remarkably large portfolio value. Such a large loss is funded by our own funds, i.e., capital which is an expensive source of funding compared to other peoples’ funds, i.e., debt. Therefore the estimation of VaR and similar market risk management measures inform banks and insurance firms with regards to the levels of capital they need to hold in order to have a buffer against unexpected downturns — market risk.

For our purpose we can fetch 5 stocks: Apple, Google, Microsoft, Intel and Box. We use a daily frequency for our data for the year 2018. We use the stock's daily closing prices to compute the continuously compounded returns: $\log\left(\frac{V_{t+1}}{V_{t}}\right) = \log(V_{t+1}) - \log(V_{t})$.



Let's estimate the expected returns vector, volatilities vector, correlation and variance-covariance matrices. The variance-covariance matrix is recovered from the estimated volatilities vector and correlation matrix: $\Omega = C \odot \sigma \sigma^{T}$ where $\odot$ is the Hadamard product, $C \in \mathbb{R}^{5 \times 5}$ and $\sigma \in \mathbb{R}^{5 \times 1}$. Portfolio volatility is estimated as: $w^{T}\Omega w$ where $w \in \mathbb{R}^{5 \times 1}$


We consider the 3 major methods used in market risk management, specifically for the estimation of VaR. Please note that there are multiple different methods for estimating VaR and other more coherent risk measures such as Conditional Value-at-Risk (CVaR) however we are only considering the few major ones.

## VaR: Variance-covariance method

---

The first one is the variance-covariance method and uses the estimated portfolio volatility $w^{T}\Omega w$ under the Gaussian assumption to estimate VaR. Let's assume we are attempting to estimate 1% VaR: This means that there is a 1% probability of obtaining a portfolio return of less than the VaR value. Using the variance-covariance approach the calculation is: $\left[\left(w^{T}\Omega w\right) \mathcal{N}^{-1}(1\%)\right] + w^{T}\mu$, where $\mu \in \mathbb{R}^{5 \times 1}$ is the expected returns vector.



## VaR: Historical simulation method
The second method is a non-parametric approach where we sample with replacement from the historical data to estimate a portfolio returns distribution. The 1% VaR is simply the appropriate quantile from this sampled portfolio returns distribution.



## VaR: Monte Carlo method

---

The third method is Monte Carlo sampling from a multidimensional Gaussian distribution using the aforementioned $mu$ and $\Omega$ parameters. Finally the 1% VaR is simply the appropriate quantile from this sampled portfolio returns distribution.



## VaR: Estimates
The VaR estimates from the aforementioned 3 market risk management methods commonly used in banking are as follows:



| VaR Method    | 1% VaR | 
| :------------- |-------------:|
| Variance-covariance | -2.87% | 
| Historical simulation | -3.65%  |
| Monte Carlo simulation | -2.63%  |

## Bidirectional Generative Adversarial Network (BiGAN)

---

The 2 main components to a Generative Adversarial Network (GAN) are the generator and the discriminator. These 2 components play an adversarial game against each other. In doing so the generator learns how to create realistic synthetic samples from noise, i.e., the latent space $z$, while the discriminator learns how to distinguish between a real sample and a synthetic sample. 

BiGAN extends GAN by adding a third component: The encoder, which learns to map from data space $x$ to the latent space $z$. The objective of the generator remains the same while the objective of the discriminator is altered to classify between a real sample and a synthetic sample and additionally between a real encoding, i.e., given by the encoder, and a synthetic encoding, i.e., a sample from the latent space $z$.

### Generator

---

Assume that we have a prior belief on where the latent space $z$ lies: $p_{Z}(z)$. Given a draw from this latent space the generator $G$, a deep learner parameterized by $\theta_{G}$, outputs a synthetic sample.

$$
G(z|\theta_{G}): z \rightarrow x_{synthetic}
$$ 

### Encoder

---

This can be shown to be an inverse of the generator. Given a draw from the data space the encoder $E$, a deep learner parameterized by $\theta_{E}$, outputs a real encoding.

$$
E(x|\theta_{E}): x \rightarrow z
$$ 

### Discriminator

---

The discriminator $D$ is a deep learner parameterized by $\theta_{D}$ and it aims to classify if a sample is real or synthetic, i.e., if a sample is from the real data distribution,

$$
p_{X}(x)
$$ 

or the synthetic data distribution.

$$
p_{G}(x|z)
$$

Additionally it aims to classify whether an encoding is real,

$$
p_{E}(z|x)
$$

or synthetic.

$$
p_{Z}(z) 
$$

Let us denote the discriminator $D$ as follows.

$$
D(\{x, z\}|\theta_{D}): \{x, z\} \rightarrow [0, 1]
$$ 

We assume that the positive examples are real, i.e., $\{x, E(x|\theta_{E})\}$ while the negative examples are synthetic, i.e., $\{G(z|\theta_{G}), z\}$. 

### Optimal discriminator, encoder and generator

---

The BiGAN has the following objective function, similar to the GAN.

$$
\min_{G(z|\theta_{G}), E(x|\theta_{E})} \max_{D(\{x, z\}|\theta_{D})} V(D(\{x, z\}|\theta_{D}), G(z|\theta_{G}), E(x|\theta_{E}))
$$

\begin{align*}
V(D(\{x, z\}|\theta_{D}), G(z|\theta_{G}), E(x|\theta_{E})) &= \mathbb{E}_{x \sim p_{X}(x)} \mathbb{E}_{z \sim p_{E}(z|x)} \log\left[{D(\{x, z\}|\theta_{D})}\right] + \mathbb{E}_{z \sim p_{Z}(z)} \mathbb{E}_{x \sim p_{G}(x|z)} \log\left[{1-D(\{x, z\}|\theta_{D})}\right] \\
&= \int_{x} p_{X}(x) \int_{z} p_{E}(z|x) \log\left[{D(\{x, z\}|\theta_{D})}\right] dz dx + \int_{z} p_{Z}(z) \int_{x} p_{G}(x|z) \log\left[{1 - D(\{x, z\}|\theta_{D})}\right] dx dz \\
&= \int_{\{x, z\}} p_{X}(x) p_{E}(z|x) \log\left[{D(\{x, z\}|\theta_{D})}\right] d\{x, z\} + \int_{\{x, z\}} p_{Z}(z) p_{G}(x|z) \log\left[{1 - D(\{x, z\}|\theta_{D})}\right] d\{x, z\} \\
&= \int_{\omega:=\{x, z\}} \underbrace{p_{EX}(\omega) \log\left[{D(\omega|\theta_{D})}\right] + p_{GZ}(\omega) \log\left[{1 - D(\omega|\theta_{D})}\right]}_{J(D(\omega|\theta_{D}))} d\omega \\
\end{align*}

Let us take a closer look at the discriminator's objective function for a sample $\omega$.

\begin{align*}
J(D(\omega|\theta_{D})) &= p_{EX}(\omega) \log{D(\omega|\theta_{D})} + p_{GZ}(\omega) \log{(1 - D(\omega|\theta_{D}))} \\
\frac{\partial J(D(\omega|\theta_{D}))}{\partial D(\omega|\theta_{D})} &= \frac{p_{EX}(\omega)}{D(\omega|\theta_{D})} - \frac{p_{GZ}(\omega)}{(1 - D(\omega|\theta_{D}))} \\
0 &= \frac{p_{EX}(\omega)}{D^\ast(\omega|\theta_{D^\ast})} - \frac{p_{GZ}(\omega)}{(1 - D^\ast(\omega|\theta_{D^\ast}))} \\
p_{EX}(\omega)(1 - D^\ast(\omega|\theta_{D^\ast})) &= p_{GZ}(\omega)D^\ast(\omega|\theta_{D^\ast}) \\
p_{EX}(\omega) - p_{EX}(\omega)D^\ast(\omega|\theta_{D^\ast})) &= p_{GZ}(\omega)D^\ast(\omega|\theta_{D^\ast}) \\
p_{GZ}(\omega)D^\ast(\omega|\theta_{D^\ast}) + p_{EX}(\omega)D^\ast(\omega|\theta_{D^\ast})) &= p_{EX}(\omega) \\
D^\ast(\omega|\theta_{D^\ast}) &= \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)} 
\end{align*}

We have found the optimal discriminator given a generator and an encoder. Let us focus now on the generator and encoder's objective function which is essentially to minimize the discriminator's objective function.

\begin{align*}
J(G(z|\theta_{G}), E(x|\theta_{E})) &= \mathbb{E}_{\omega \sim p_{EX}(\omega)} \log{D^\ast(\omega|\theta_{D^\ast})} + \mathbb{E}_{\omega \sim p_{GZ}(\omega)} \log{(1 - D^\ast(\omega|\theta_{D^\ast}))} \\
&= \mathbb{E}_{\omega \sim p_{EX}(\omega)} \log{\bigg( \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) + \mathbb{E}_{\omega \sim p_{GZ}(\omega)} \log{\bigg(1 - \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} \\
&= \mathbb{E}_{\omega \sim p_{EX}(\omega)} \log{\bigg( \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) + \mathbb{E}_{\omega \sim p_{GZ}(\omega)} \log{\bigg(\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} \\
&= \int_{\omega} p_{EX}(\omega) \log{\bigg( \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) d\omega + \int_{\omega} p_{GZ}(\omega) \log{\bigg(\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} d\omega
\end{align*}

We will note the Kullback–Leibler (KL) divergences in the above objective function for the generator and encoder.

$$
D_{KL}(P||Q) = \int_{x} p(x) \log\bigg(\frac{p(x)}{q(x)}\bigg) dx
$$

Recall the definition of a $\lambda$ divergence.

$$
D_{\lambda}(P||Q) = \lambda D_{KL}(P||\lambda P + (1 - \lambda) Q) + (1 - \lambda) D_{KL}(Q||\lambda P + (1 - \lambda) Q)
$$

If $\lambda$ takes the value of 0.5 this is then called the Jensen-Shannon (JS) divergence. This divergence is symmetric and non-negative.

$$
D_{JS}(P||Q) = 0.5 D_{KL}\bigg(P\bigg|\bigg|\frac{P + Q}{2}\bigg) + 0.5 D_{KL}\bigg(Q\bigg|\bigg|\frac{P + Q}{2}\bigg)
$$

Keeping this in mind let us take a look again at the objective function of the generator and the encoder.

\begin{align*}
J(G(z|\theta_{G}), E(x|\theta_{E})) &= \int_{\omega} p_{EX}(\omega) \log{\bigg( \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) d\omega + \int_{\omega} p_{GZ}(\omega) \log{\bigg(\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} d\omega \\
&= \int_{\omega} p_{EX}(\omega) \log{\bigg(\frac{2}{2}\frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) d\omega + \int_{\omega} p_{GZ}(\omega) \log{\bigg(\frac{2}{2}\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} d\omega \\
&= \int_{\omega} p_{EX}(\omega) \log{\bigg(\frac{1}{2}\frac{1}{0.5}\frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}} \bigg) d\omega + \int_{\omega} p_{GZ}(\omega) \log{\bigg(\frac{1}{2}\frac{1}{0.5}\frac{p_{GZ}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)}\bigg)} d\omega \\
&= \int_{\omega} p_{EX}(\omega) \bigg[ \log(0.5) + \log{\bigg(\frac{p_{EX}(\omega)}{0.5 (p_{EX}(\omega) + p_{GZ}(\omega))}} \bigg) \bigg] d\omega \\ &+ \int_{\omega} p_{GZ}(\omega) \bigg[\log(0.5) + \log{\bigg(\frac{p_{GZ}(\omega)}{0.5 (p_{EX}(\omega) + p_{GZ}(\omega))}\bigg) \bigg] } d\omega \\
&= \log\bigg(\frac{1}{4}\bigg) + \int_{\omega} p_{EX}(\omega) \bigg[\log{\bigg(\frac{p_{EX}(\omega)}{0.5 (p_{EX}(\omega) + p_{GZ}(\omega))}} \bigg) \bigg] d\omega \\ 
&+ \int_{\omega} p_{GZ}(\omega) \bigg[\log{\bigg(\frac{p_{GZ}(\omega)}{0.5 (p_{EX}(\omega) + p_{GZ}(\omega))}\bigg) \bigg] } d\omega \\
&= -\log(4) + D_{KL}\bigg(P_{EX}\bigg|\bigg|\frac{P_{EX} + P_{GZ}}{2}\bigg) + D_{KL}\bigg(P_{GZ}\bigg|\bigg|\frac{P_{EX} + P_{GZ}}{2}\bigg) \\
&= -\log(4) + 2 \bigg(0.5 D_{KL}\bigg(P_{EX}\bigg|\bigg|\frac{P_{EX} + P_{GZ}}{2}\bigg) + 0.5 D_{KL}\bigg(P_{GZ}\bigg|\bigg|\frac{P_{EX} + P_{GZ}}{2}\bigg)\bigg) \\
&= -\log(4) + 2D_{JS}(P_{EX}||P_{GZ}) 
\end{align*}

It is clear from the objective function of the generator and encoder above that the global minimum value attained is $-\log(4)$ which occurs when the following holds.

$$
P_{EX}=P_{GZ}
$$

When the above holds the Jensen-Shannon divergence, i.e., $D_{JS}(P_{EX}||P_{GZ})$, will be zero. Hence we have shown that the optimal solution is as follows.

$$
P_{EX}=P_{GZ}
$$

Given the above result we can prove that the optimal discriminator will be $\frac{1}{2}$.

\begin{align*}
D^\ast(\omega|\theta_{D^\ast}) &= \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{GZ}(\omega)} \\
 &= \frac{p_{EX}(\omega)}{p_{EX}(\omega) + p_{EX}(\omega)} \\
 &= \frac{p_{EX}(\omega)}{2p_{EX}(\omega)} \\
 &= \frac{1}{2} \\
\end{align*}

### Optimal encoder and generator are inverse functions of each other

---

At the optimal generator and encoder we can show that the generator and encoder are inverse functions of each other. Recall from earlier the definitions of the generator and the encoder.

$$
G(z|\theta_{G}): z \rightarrow x_{synthetic}
$$ 

$$
E(x|\theta_{E}): x \rightarrow z
$$ 

At this point the optimal discriminator is $\frac{1}{2}$, i.e., the discriminator cannot effectively differentiate between real and synthetic data as the synthetic data is realistic. Remember that at this point the likelihood would have been implicitly maximized such that any samples taken from the synthetic distribution should be similar to those taken from the real distribution. In short, if optimality of the generator, encoder and discriminator holds then the synthetic data should look similar, or rather be the same, as the real data. Keeping this important point in mind let's slightly re-write the optimal generator and encoder functions.

$$
G^\ast(z|\theta_{G^\ast}): z \rightarrow x
$$ 

$$
E^\ast(x|\theta_{E^\ast}): x \rightarrow z
$$ 

Recall further that the following holds at the optimal generator and encoder.

\begin{align*}
P_{EX} &= \int_{x} p_{X}(x) \int_{z=E^\ast(x|\theta_{E^\ast})} p_{E^\ast}(z|x) dz dx \\
\end{align*}

In the above please note the following; note also that we make the assumption that the generator is not an inverse function of the encoder for providing a proof by contradiction.

\begin{align*}
z&=E^\ast(x|\theta_{E^\ast}) \\
x&\neq G^\ast(E^\ast(x|\theta_{E^\ast})|\theta_{G^\ast}) \\
\end{align*}

Recall that optimality condition of the generator and encoder.

\begin{align*}
P_{EX} &= P_{GZ} \\
P_{GZ} &= \int_{z} p_{Z}(z) \int_{x=G^\ast(z|\theta_{G^\ast})} p_{G^\ast}(x|z) dx dz \\
\end{align*}

In the above please note the following.

\begin{align*}
x&=G^\ast(z|\theta_{G^\ast}) \\
z&=E^\ast(x|\theta_{E^\ast}) \\
z&=E^\ast(G^\ast(z|\theta_{G^\ast})|\theta_{E^\ast}) \\
G^\ast(z|\theta_{G^\ast})&=G^\ast(E^\ast(G^\ast(z|\theta_{G^\ast})|\theta_{E^\ast})|\theta_{G^\ast}) \\
\end{align*}

If optimality holds then the following holds as shown above.

$$
G^\ast(z|\theta_{G^\ast})=G^\ast(E^\ast(G^\ast(z|\theta_{G^\ast})|\theta_{E^\ast})|\theta_{G^\ast})
$$

However since we assumed that the generator is not an inverse function of the encoder then the above conditions cannot hold thereby violating the optimality condition.

\begin{align*}
x&\neq G^\ast(E^\ast(x|\theta_{E^\ast})|\theta_{G^\ast}) \\
G^\ast(z|\theta_{G^\ast})&\neq G^\ast(E^\ast(G^\ast(z|\theta_{G^\ast})|\theta_{E^\ast})|\theta_{G^\ast}) \\
\end{align*}

Therefore we have shown by contradiction that under optimality of the generator and encoder the generator is an inverse function of the encoder.

\begin{align*}
x&=G^\ast(E^\ast(x|\theta_{E^\ast})|\theta_{G^\ast}) \\
\end{align*}

The same arguments made above can be shown for the encoder being the inverse of the generator.

\begin{align*}
P_{GZ} &= \int_{z} p_{Z}(z) \int_{x=G^\ast(z|\theta_{G^\ast})} p_{G^\ast}(x|z) dx dz \\
\end{align*}

In the above please note the following; note also that we make the assumption that the encoder is not an inverse function of the generator for providing a proof by contradiction.

\begin{align*}
x&=G^\ast(z|\theta_{G^\ast}) \\
z&\neq E^\ast(G^\ast(z|\theta_{G^\ast})|\theta_{E^\ast}) \\
\end{align*}

Recall that optimality condition of the generator and encoder.

\begin{align*}
P_{EX} &= P_{GZ} \\
P_{EX} &= \int_{x} p_{X}(x) \int_{z=E^\ast(x|\theta_{E^\ast})} p_{E^\ast}(z|x) dz dx \\
\end{align*}

In the above please note the following.

\begin{align*}
z&=E^\ast(x|\theta_{E^\ast}) \\
x&=G^\ast(z|\theta_{G^\ast}) \\
x&=G^\ast(E^\ast(x|\theta_{E^\ast})|\theta_{G^\ast}) \\
E^\ast(x|\theta_{E^\ast})&=E^\ast(G^\ast(E^\ast(x|\theta_{E^\ast})|\theta_{G^\ast})|\theta_{E^\ast}) \\
\end{align*}

If optimality holds then the following holds as shown above.

$$
E^\ast(x|\theta_{E^\ast})=E^\ast(G^\ast(E^\ast(x|\theta_{E^\ast})|\theta_{G^\ast})|\theta_{E^\ast})
$$

However since we assumed that the encoder is not an inverse function of the generator then the above conditions cannot hold thereby violating the optimality condition.

\begin{align*}
z&\neq E^\ast(G^\ast(z|\theta_{G^\ast})|\theta_{E^\ast}) \\
E^\ast(x|\theta_{E^\ast})&\neq E^\ast(G^\ast(E^\ast(x|\theta_{E^\ast})|\theta_{G^\ast})|\theta_{E^\ast}) \\
\end{align*}

Therefore we have shown by contradiction that under optimality of the generator and encoder the encoder is an inverse function of the generator.

\begin{align*}
z&= E^\ast(G^\ast(z|\theta_{G^\ast})|\theta_{E^\ast}) \\
\end{align*}

Therefore we have shown that the optimal encoder and generator are inverse functions of each other via proof by contradiction: If they were not inverse functions of each other then it would violate the optimality condition for the encoder and generator, i.e.,  $P_{EX} = P_{GZ}$.



In [1]:
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()
df_full = pdr.get_data_yahoo(["AAPL","GOOGL","MSFT","INTC","BOX"], start="2016-01-01", end="2017-01-01").reset_index()
df_full.head()

[*********************100%***********************]  5 of 5 downloaded


Date        Open         ...   Volume                    
                    AAPL    BOX  ...    GOOGL      INTC      MSFT
0 2016-01-04  102.610001  13.46  ...  3369100  27882200  53778000
1 2016-01-05  105.750000  14.25  ...  2260800  16709500  34079700
2 2016-01-06  100.559998  13.18  ...  2410300  25491300  39518900
3 2016-01-07   98.680000  12.41  ...  3156600  37680500  56564900
4 2016-01-08   98.550003  12.31  ...  2375300  29953800  48754000

[5 rows x 31 columns]

In [2]:
df_full = df_full.set_index('Date')
df_full = df_full['Adj Close']
df_full = df_full.pct_change().ffill().bfill()
df_full.head()

,AAPL,BOX,GOOGL,INTC,MSFT
Date,,,,,
2016-01-04,-0.025059,-0.072423,0.002752,-0.004707,0.004562
2016-01-05,-0.025059,-0.072423,0.002752,-0.004707,0.004562
2016-01-06,-0.019570,-0.047297,-0.002889,-0.022170,-0.018165
2016-01-07,-0.042204,-0.040977,-0.024140,-0.037485,-0.034783
2016-01-08,0.005288,-0.046015,-0.013617,-0.010364,0.003067


In [3]:
from PIL import Image

from six.moves import range

import os
import math
import inspect
import sys
import importlib

import numpy as np

import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import keras
from keras import backend as bkend
from keras import layers
from keras.layers import Input, Dense, BatchNormalization, Dropout, Flatten, convolutional, pooling, Reshape, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras import metrics
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.utils.generic_utils import Progbar

import tensorflow as tf
from tensorflow.python.client import device_lib

import matplotlib.pyplot as plt

from plotnine import *
import plotnine

get_ipython().magic("matplotlib inline")

os.environ["KERAS_BACKEND"] = "tensorflow"
importlib.reload(bkend)

print(device_lib.list_local_devices())

ret_data = df_full.copy()
mean = ret_data.apply(func=np.mean, axis=0)
std = ret_data.apply(func=np.std, axis=0)
ret_data -= mean
ret_data /= std

Using TensorFlow backend.
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9899822543429676215
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7926420924738207938
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15166282187119119058
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11326753997
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12995622532743248017
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [4]:
# License
# Copyright 2018 Hamaad Musharaf Shah
# Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
# http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# BiGAN implementation inspired from here: https://github.com/eriklindernoren/Keras-GAN
# I believe I can re-write it slightly differently. On my to-do list.
class BiGAN(BaseEstimator,
            TransformerMixin):
    def __init__(self,
                 z_size=None,
                 iterations=None,
                 batch_size=None):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        
        for arg, val in values.items():
            setattr(self, arg, val)
            
        # Build the discriminator.
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(optimizer=RMSprop(lr=0.0002, 
                                                     clipvalue=1.0,
                                                     decay=1e-8),
                                   loss="binary_crossentropy",
                                   metrics=["accuracy"])

        # Build the generator to fool the discriminator.
        # Freeze the discriminator here.
        self.discriminator.trainable = False
        self.generator = self.build_generator()
        self.encoder = self.build_encoder()
        
        noise = Input(shape=(self.z_size, ))
        generated_data = self.generator(noise)
        fake = self.discriminator([noise, generated_data])

        real_data = Input(shape=(5,))
        encoding = self.encoder(real_data)
        valid = self.discriminator([encoding, real_data])

        # Set up and compile the combined model.
        # Trains generator to fool the discriminator.
        self.bigan_generator = Model([noise, real_data], [fake, valid])
        self.bigan_generator.compile(loss=["binary_crossentropy", "binary_crossentropy"],
                                     optimizer=RMSprop(lr=0.0004, 
                                                       clipvalue=1.0,
                                                       decay=1e-8))
 
    def fit(self,
            X,
            y=None):
        num_train = X.shape[0]
        start = 0
        
        # Adversarial ground truths.
        valid = np.ones((self.batch_size, 1)) 
        fake = np.zeros((self.batch_size, 1))        
        
        for step in range(self.iterations):
            # Generate a new batch of noise...
            noise = np.random.uniform(low=-1.0, high=1.0, size=(self.batch_size, self.z_size))
            # ...and generate a batch of synthetic returns data.
            generated_data = self.generator.predict(noise)
            
            # Get a batch of real returns data...
            stop = start + self.batch_size
            real_batch = X[start:stop]
            # ...and encode them.
            encoding = self.encoder.predict(real_batch)

            # Train the discriminator.
            d_loss_real = self.discriminator.train_on_batch([encoding, real_batch], valid)
            d_loss_fake = self.discriminator.train_on_batch([noise, generated_data], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator.
            g_loss = self.bigan_generator.train_on_batch([noise, real_batch], [valid, fake])
            
            start += self.batch_size
            if start > num_train - self.batch_size:
                start = 0
            
            if step % 100 == 0:
                # Plot the progress.
                print("[Discriminator loss: %f, Discriminator accuracy: %.2f%%] [Generator loss: %f]" % (d_loss[0], 100 * d_loss[1], g_loss[0]))
                
        return self

    def transform(self,
                  X):
        return self.feature_extractor.predict(X)

    def build_encoder(self):
        encoder_input = Input(shape=(5,))

        encoder_model = Dense(units=100)(encoder_input)
        encoder_model = LeakyReLU(alpha=0.2)(encoder_model)
        encoder_model = BatchNormalization()(encoder_model)
        encoder_model = Dense(units=100)(encoder_model)
        encoder_model = LeakyReLU(alpha=0.2)(encoder_model)
        
        encoder_output = Dense(units=self.z_size, activation="tanh")(encoder_model)
        
        self.feature_extractor = Model(encoder_input, encoder_output)
        
        return Model(encoder_input, encoder_output)
    
    def build_generator(self):
        # We will map z, a latent vector, to continuous returns data space (..., 5).
        latent = Input(shape=(self.z_size,))

        # This produces a (..., 100) shaped tensor.
        generator_model = Dense(units=100, activation="elu")(latent)
        generator_model = BatchNormalization()(generator_model)
        generator_model = Dense(units=100, activation="elu")(generator_model)
        generator_model = BatchNormalization()(generator_model)

        generator_output = Dense(units=5, activation="linear")(generator_model)
        
        return Model(latent, generator_output)
    
    def build_discriminator(self):
        z = Input(shape=(self.z_size,))
        ret_data = Input(shape=(5,))
        discriminator_inputs = concatenate([z, ret_data], axis=1)

        discriminator_model = Dense(units=100)(discriminator_inputs)
        discriminator_model = LeakyReLU(alpha=0.2)(discriminator_model)
        discriminator_model = Dropout(rate=0.5)(discriminator_model)

        discriminator_output = Dense(units=1, activation="sigmoid")(discriminator_model)
        
        return Model([z, ret_data], discriminator_output)

z_size = 10
bigan = BiGAN(z_size=z_size,
              batch_size=100,
              iterations=10000)

bigan.fit(X=ret_data)

n_sim = 1000
noise = np.random.uniform(low=-1.0, high=1.0, size=(n_sim, z_size))
x = np.zeros(shape=(n_sim, 5))
x_mean = np.zeros(shape=n_sim)
for i, xi in enumerate(noise):
    x[i, :] = (bigan.generator.predict(x=np.array([xi]))[0] * std) + mean
    x_mean[i] = np.average(a=x[i, :])

act_mean = np.zeros(shape=ret_data.shape[0])
for i in range(ret_data.shape[0]):
    act_mean[i] = np.average(a=(ret_data.iloc[i] * std) + mean)
    
plotnine.options.figure_size = (12, 9)
plot = ggplot(pd.melt(pd.concat([pd.DataFrame(x_mean, columns=["BiGAN Portfolio Returns Distribution"]).reset_index(drop=True),
                                 pd.DataFrame(act_mean, columns=["Actual Portfolio Returns Distribution"]).reset_index(drop=True)],
                                axis=1))) + \
geom_density(aes(x="value",
                 fill="factor(variable)"), 
             alpha=0.5,
             color="black") + \
geom_point(aes(x="value",
               y=0,
               fill="factor(variable)"), 
           alpha=0.5, 
           color="black") + \
xlab("Portfolio returns") + \
ylab("Density") + \
ggtitle("Trained Bidirectional Generative Adversarial Network (BiGAN) Portfolio Returns") + \
theme_matplotlib()
plot.save(filename="trained_bigan_sampler.png")

untrained_bigan = BiGAN(z_size=z_size,
                        batch_size=100,
                        iterations=10000)

untrained_x = np.zeros(shape=(n_sim, 5))
untrained_x_mean = np.zeros(shape=n_sim)
for i, xi in enumerate(noise):
    untrained_x[i, :] = (untrained_bigan.generator.predict(x=np.array([xi]))[0] * std) + mean
    untrained_x_mean[i] = np.average(a=untrained_x[i, :])

plotnine.options.figure_size = (12, 9)
plot = ggplot(pd.melt(pd.concat([pd.DataFrame(untrained_x_mean, columns=["BiGAN Portfolio Returns Distribution"]).reset_index(drop=True),
                                 pd.DataFrame(act_mean, columns=["Actual Portfolio Returns Distribution"]).reset_index(drop=True)],
                                axis=1))) + \
geom_density(aes(x="value",
                 fill="factor(variable)"), 
             alpha=0.5,
             color="black") + \
geom_point(aes(x="value",
               y=0,
               fill="factor(variable)"), 
           alpha=0.5, 
           color="black") + \
xlab("Portfolio returns") + \
ylab("Density") + \
ggtitle("Untrained Bidirectional Generative Adversarial Network (BiGAN) Portfolio Returns") + \
theme_matplotlib()
plot.save(filename="untrained_bigan_sampler.png")

print("The VaR at 1%% estimate given by the BiGAN: %.2f%%" % (100 * np.percentile(a=x_mean, axis=0, q=1)))

W0710 01:38:12.955640 140333317891968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0710 01:38:12.961388 140333317891968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 01:38:12.976481 140333317891968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0710 01:38:12.995869 140333317891968 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0710 01:38:13.008757 

[Discriminator loss: 0.694480, Discriminator accuracy: 51.50%] [Generator loss: 1.638100]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Discriminator loss: 0.820894, Discriminator accuracy: 38.00%] [Generator loss: 1.255132]
[Discriminator loss: 0.764588, Discriminator accuracy: 47.00%] [Generator loss: 1.353739]
[Discriminator loss: 0.722767, Discriminator accuracy: 46.50%] [Generator loss: 1.425732]
[Discriminator loss: 0.736330, Discriminator accuracy: 46.00%] [Generator loss: 1.371265]
[Discriminator loss: 0.685728, Discriminator accuracy: 51.50%] [Generator loss: 1.413202]
[Discriminator loss: 0.700723, Discriminator accuracy: 48.50%] [Generator loss: 1.361335]
[Discriminator loss: 0.697648, Discriminator accuracy: 51.50%] [Generator loss: 1.382622]
[Discriminator loss: 0.727550, Discriminator accuracy: 46.50%] [Generator loss: 1.411593]
[Discriminator loss: 0.723220, Discriminator accuracy: 42.50%] [Generator loss: 1.399636]
[Discriminator loss: 0.707107, Discriminator accuracy: 50.00%] [Generator loss: 1.369449]
[Discriminator loss: 0.717697, Discriminator accuracy: 51.00%] [Generator loss: 1.426095]
[Discrimin

/usr/local/lib/python3.6/dist-packages/plotnine/ggplot.py:706: UserWarning: Saving 12 x 9 in image.
  from_inches(height, units), units))
/usr/local/lib/python3.6/dist-packages/plotnine/ggplot.py:707: UserWarning: Filename: trained_bigan_sampler.png
  warn('Filename: {}'.format(filename))
/usr/local/lib/python3.6/dist-packages/plotnine/layer.py:517: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  return not cbook.iterable(value) and (cbook.is_numlike(value) or
/usr/local/lib/python3.6/dist-packages/plotnine/layer.py:360: UserWarning: stat_density : Removed 748 rows containing non-finite values.
  data = self.stat.compute_layer(data, params, layout)
/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/usr/local/lib/python3.6/dist-packages/matplotlib/__init__

The VaR at 1% estimate given by the BiGAN: -2.16%


## Conclusion

Before we conclude the article let's have a look at the portfolio returns distribution sampled from an untrained BiGAN. 

![](untrained_bigan_sampler.png)

It is clear from the above graph that the untrained BiGAN's sampled portfolio returns distribution is remarkably different from the actual portfolio returns distribution. This is, as we can imagine, to be expected.

Contrast this with a trained BiGAN: The following graph will clearly show the value of GAN type models for market risk management as we have arrived at this learnt portfolio returns distribution without having to rely on a possibly incorrect assumption with regards to the actual portfolio returns distribution such as a multidimensional Gaussian distribution.

Note that we perhaps should use an evolutionary algorithm or a reinforcement learner to automatically learn the appropriate GAN or BiGAN architecture: Perhaps that shall be a topic for a future article.

![](trained_bigan_sampler.png)

Finally we update the VaR estimate table for using different market risk management methods as below. We can see that the VaR estimate provided by the BiGAN is similar, if not exactly the same, to the ones provided by the other market risk management methods. This provides us with a good sanity check with regards to using the BiGAN for market risk management in that it provides competitive results with respect to well established existing market risk management methods.

| VaR Method    | 1% VaR | 
| :------------- |-------------:|
| Variance-covariance | -1.37% | 
| Historical simulation | -1.65%  |
| Monte Carlo simulation | -1.43%  |
| Bidirectional Generative Adversarial Network | -2.16%  |

The portfolio of 5 stocks we had to work with was not particularly complicated compared to potentially having portfolios where we might have derivatives or other portfolio components. Arriving at the correct portfolio returns distribution of a potentially complicated portfolio is a problem that has been shown can be solved via deep learning specifically the BiGAN. This result can be useful for market risk management and any other different problem space where we need to generate samples from a potentially complex, and perhaps unknown, distribution. 

There will potentially be a follow up article of mine where we look at a complicated backtesting scenario, i.e., validating that market risk management VaR type estimates provided by BiGAN is appropriate for future portfolio returns distributions that we have not seen, and perhaps using more complicated portfolios. The aim of this article of mine was to clearly show that a trained BiGAN can be used for market risk management VaR estimation for a given portfolio.

## References

1. Goodfellow, I., Bengio, Y. and Courville A. (2016). Deep Learning (MIT Press).
2. Geron, A. (2017). Hands-On Machine Learning with Scikit-Learn & Tensorflow (O'Reilly).
3. Kingma, D. P., and Welling M. (2014). Auto-Encoding Variational Bayes (https://arxiv.org/abs/1312.6114).
4. http://scikit-learn.org/stable/#
5. https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
6. https://stackoverflow.com/questions/42177658/how-to-switch-backend-with-keras-from-tensorflow-to-theano
7. https://blog.keras.io/building-autoencoders-in-keras.html
8. https://keras.io
9. Chollet, F. (2018). Deep Learning with Python (Manning).
10. Hull, John C. (2010). Risk Management and Financial Institutions (Pearson).
11. https://towardsdatascience.com/automatic-feature-engineering-using-deep-learning-and-bayesian-inference-application-to-computer-7b2bb8dc7351
12. https://towardsdatascience.com/automatic-feature-engineering-using-generative-adversarial-networks-8e24b3c16bf3
13. Donahue, J., Krähenbühl, P. and Darrell, T. (2017). Adversarial Feature Learning (https://arxiv.org/pdf/1605.09782).
14. https://github.com/eriklindernoren/Keras-GAN